In [1]:
import requests
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

prenset = "出席"
absent = "缺席"
leave = "請假"
url = "https://script.googleusercontent.com/macros/echo?user_content_key=CS1xGtawxafbsz_VXLceDZ8YhZJ3WLRGo3cTurMdyLFpRky5qzeXoqZBrYuJ0ElreSkNapo1i9TRXgBoQpou4xhHPHjfNUj4m5_BxDlH2jW0nuo2oDemN9CCS2h10ox_1xSncGQajx_ryfhECjZEnNGpsruz596piGbFJMaeWeZjFnhN0a9d4HwmOCyefi3ex5PPcpNsPlxGCUpGq7rwIA3gVkC6Pkbl8Md7CuyY-jH4-8QW02051A&lib=McNoStEX3P6w1EymfWUjOXqNKe58B53cM"

In [2]:
def getData(url: str):
    try:
        response =  requests.get(url)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
    except Exception as e:
        print(f"Unexpected error occurred: {e}")
    return None

In [3]:
def getDf_from_google_sheet_json(resp_json):
    df = pd.DataFrame(resp_json['data'][1:], columns=resp_json['data'][0])
    strings = df.columns
    pattern = r"\[(.*?)\]"
    result = []
    for string in strings:
        matches = re.findall(pattern, string)
        result.extend(matches)
    df.columns = list(df.columns[:2]) + result
    df['date'] = pd.to_datetime(df['日期']).dt.strftime('%Y-%m-%d')
    cols = df.columns.tolist()[2:]
    cols = [cols[-1]] + cols[:-1]
    output_df = df[cols].copy()
    return output_df

In [4]:
def getDf_present_rate(df, prenset, absent, leave):
    result = pd.DataFrame()
    for column in df.columns[2:]:
        present_count = df[column].value_counts().get(prenset, 0)
        absent_count = df[column].value_counts().get(absent, 0)
        leave_count = df[column].value_counts().get(leave, 0)
        total_count = present_count + absent_count + leave_count
        present_rate = round((present_count / total_count * 100),2) if total_count > 0 else 0
        result.loc[column, 'Present Count'] = present_count
        result.loc[column, 'Absent Count'] = absent_count
        result.loc[column, 'Leave Count'] = leave_count
        result.loc[column, 'Present Rate (%)'] = present_rate
        result = result.sort_index()
    return result

In [5]:
def filter_last_x_months(df, x):
    if x == 0:
        return df
    today = datetime.today()
    x_months_ago = today - relativedelta(months=x)
    temp_df = df.copy()
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    return df[temp_df['date'] >= x_months_ago]

In [6]:
resp = getData(url)
df = None
if resp is not None:
    df = getDf_from_google_sheet_json(resp.json())


In [7]:
filtered_df = filter_last_x_months(df,0)
result_df = getDf_present_rate(filtered_df, prenset, absent, leave)
result_df

,Present Count,Absent Count,Leave Count,Present Rate (%)
Caitlyn,0.0,0.0,0.0,0.0
Chautung,0.0,0.0,0.0,0.0
Chuilam,0.0,0.0,0.0,0.0
Heiting,0.0,0.0,0.0,0.0
SzeKi,0.0,0.0,0.0,0.0
Yannis,0.0,0.0,0.0,0.0
Yau,0.0,0.0,0.0,0.0
Yoyo,0.0,0.0,0.0,0.0


In [22]:
formated_df = result_df.style.format("{:.2f}".format)

In [23]:
formated_df

,Present Count,Absent Count,Leave Count,Present Rate (%)
Caitlyn,8.00,0.00,0.00,100.00
Chautung,8.00,0.00,0.00,100.00
Chuilam,2.00,4.00,2.00,25.00
Heiting,4.00,4.00,0.00,50.00
SzeKi,6.00,0.00,1.00,85.71
Yannis,8.00,0.00,0.00,100.00
Yau,5.00,1.00,2.00,62.50
Yoyo,6.00,1.00,1.00,75.00


In [24]:
import pandas as pd

# Create a sample DataFrame
data = {
    "Name": ["Alice", "Bob", "Charlie"],
    "Age": [25, 30, 35],
    "Score": [85, 90, 88]
}
df = pd.DataFrame(data)

# Sort the DataFrame by a specific column name
column_name = "Name"  # Specify the column name you want to sort by
sorted_df = df.sort_values(by=column_name)

print(sorted_df)

      Name  Age  Score
0    Alice   25     85
1      Bob   30     90
2  Charlie   35     88


In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {
    "Name": ["Alice", "Bob", "Charlie"],
    "Age": [25, 30, 35],
    "Score": [85, 90, 88]
}
df = pd.DataFrame(data)

# Sort the DataFrame columns alphabetically by column name
df = df.reindex(sorted(df.columns), axis=1)

print(df)

   Age     Name  Score
0   25    Alice     85
1   30      Bob     90
2   35  Charlie     88


In [33]:
x_months = [i for i in range(0,13,3)]
x_months =  x_months[1:] + [x_months[0]]
print(x_months)

[3, 6, 9, 12, 0]
